In [2]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [3]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import FinanceDataReader as fdr

In [5]:
import koreanize_matplotlib

In [6]:
import datetime
from dateutil.relativedelta import relativedelta

In [7]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [9]:
# n개월 전 날짜 계산 함수(개월단위)
def calculate_start_date(months_ago, end_date):
    start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') - relativedelta(months=months_ago)
    return start_date.strftime('%Y-%m-%d')

# 오늘 날짜 구하기
today = datetime.datetime.today()
today_str = today.strftime('%Y-%m-%d')

In [8]:
kospi = pd.read_csv('kospi_add_sector_2024-12-28.csv')
kosdaq = pd.read_csv('kosdaq_add_sector_2024-12-28.csv')

In [ ]:
df = pd.read_csv('sector_by_stock_list_df.csv')

In [ ]:
# fdr.StockListing으로 불러온 df - KOSPI와 KOSDAQ에 'Sector' 열 추가 (초기값 None)
# 처음 데이터를 가공할때 사용

for index in range(len(df)):
    stock_name = df['종목명'][index]
    sector_name = df['업종명'][index]
    market = df['코스닥 여부'][index]
    
    if market:  # KOSDAQ인 경우
        if stock_name in kosdaq['Name'].values:
            kosdaq.loc[kosdaq['Name'] == stock_name, 'Sector'] = sector_name
    else:  # KOSPI인 경우
        if stock_name in kospi['Name'].values:
            kospi.loc[kospi['Name'] == stock_name, 'Sector'] = sector_name

# 오늘 날짜
today = datetime.datetime.today()
today = today.strftime('%Y-%m-%d')

# CSV 파일로 저장
kospi.to_csv(f'kospi_add_sector_{today}.csv', index=False, encoding='utf-8-sig')
kosdaq.to_csv(f'kosdaq_add_sector_{today}.csv', index=False, encoding='utf-8-sig')

In [12]:
def combined_stock_analysis(market='KOSPI', month_ago=1, end_date=today_str):
    """
    주어진 시장(KOSPI 또는 KOSDAQ)의 각 종목에 대한 변동성과 수익률을 계산하는 함수.

    Parameters:
    market (str): 'KOSPI' 또는 'KOSDAQ' 중 하나를 선택하여 해당 시장의 종목을 대상으로 분석.
    month_ago (int): 계산할 시작 날짜로부터 몇 개월 전부터 수익률을 계산할 것인지 지정.
    end_date (str, optional): 데이터 조회의 종료 날짜 (형식: 'YYYY-MM-DD'). 기본값은 오늘 날짜.

    Returns:
    pd.DataFrame: 각 종목의 코드, 이름, 변동성, 총 수익률, 평균 수익률 정보를 포함하는 데이터프레임.
    
    예외 처리:
    데이터 로드 중 오류가 발생하면 해당 종목에 대한 오류 메시지를 출력.
    """
    
    start_date = calculate_start_date(month_ago, end_date)
    results = []  # 결과를 저장할 리스트

    # 시장 데이터 선택
    if market == 'KOSPI':
        sector_data = kospi
    elif market == 'KOSDAQ':
        sector_data = kosdaq
    else:
        raise ValueError(" 'KOSPI', 'KOSDAQ' 두 시장만 지원하는 기능입니다.")
    
    # 'Code'를 인덱스로 설정
    sector_data = sector_data.set_index('Code')

    # 각 종목의 데이터를 가져와 변동성과 수익률 계산
    for index, row in sector_data.iterrows():
        ticker = index
        try:
            # 각 종목의 데이터 가져오기
            data = fdr.DataReader(ticker, start=start_date, end=end_date)
            if data.empty:
                print(f"종목 {ticker}의 데이터가 없습니다. 건너뜁니다.")
                continue
            
            # 수익률 계산
            
            data['Returns'] = data['Close'].pct_change() * 100  # 수익률을 퍼센트로 변환
            data.dropna(inplace=True)  # NaN 값 제거
            
            if len(data) == 1:  # 상장이후 일일수익률 데이터가 단 하나라  수익률 계산이 의미가 없는경우
                print(f"종목 {ticker}은 일일수익률 데이터가 하나라, 수익률 계산이 의미가 없기때문에 건너뜁니다.")
                continue
      
            if len(data) == 0: # 상장이후 종가가 하나라 수익률 계산 자체가 되지 않는경우
                print(f"종목 {ticker}의 데이터가 충분하지 않습니다. 건너뜁니다.")
                continue
            
            # 변동성 계산 (표준편차)
            volatility = data['Returns'].std()
            
            # 총 수익률 계산
            total_return = (data['Close'].iloc[-1] - data['Close'].iloc[0]) / data['Close'].iloc[0] * 100
            
            # 평균 수익률 계산산
            avg_return = data['Returns'].mean()
            
            # 결과 저장
            results.append({
                'Code': ticker,                 # 종목코드
                'Name': row['Name'],            # 종목명
                'Volatility': volatility,       # 변동성(%)
                'TotalReturn': total_return,    # 총수익률(%)
                'AvgReturn': avg_return,        # 평균수익률(%)      
            })

        except Exception as e:
            print(f"{ticker} 데이터 오류: {e}")

    # 결과 DataFrame 생성
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"new_{market}_df.csv",index=False, encoding='utf-8-sig')

    return results_df


In [13]:
kospi_df = combined_stock_analysis(market='KOSPI')
kospi_df.head()

,Code,Name,Volatility,TotalReturn,AvgReturn
0,005930,삼성전자,1.578523,0.186567,-0.036956
1,000660,SK하이닉스,2.346059,9.886650,0.487213
2,373220,LG에너지솔루션,2.568578,-11.111111,-0.451771
3,207940,삼성바이오로직스,1.327286,-3.810505,-0.217512
4,005380,현대차,2.167478,-0.462963,-0.063012


In [14]:
kosdaq_df = combined_stock_analysis(market='KOSDAQ')
kosdaq_df.head()

종목 387570은 일일수익률 데이터가 하나라, 수익률 계산이 의미가 없기때문에 건너뜁니다.
종목 432980은 일일수익률 데이터가 하나라, 수익률 계산이 의미가 없기때문에 건너뜁니다.


,Code,Name,Volatility,TotalReturn,AvgReturn
0,196170,알테오젠,4.780943,-0.331126,0.486394
1,247540,에코프로비엠,2.686852,-21.454545,-1.140209
2,028300,HLB,2.654853,-1.540616,-0.077903
3,086520,에코프로,2.214570,-24.603175,-1.485840
4,141080,리가켐바이오,4.850454,5.924413,0.543159


In [ ]:
# 개선된 함수 - 데이터 전처리
# 반복문의 사용을 줄이고, 중복 코드를 줄인 형태
# merge를 통해 두 df를 결합하였기 때문에, 대량의 데이터 소화시 성능향상이 기대

def add_sector_info(*, market=None, local_df=None, local_market=None) :
    
    '''
    주어진 시장(KOSPI 또는 KOSDAQ)의 종목 데이터에 Sector, Volume, Marcap, Amount 정보를 추가합니다.

    이 함수는 KOSPI 또는 KOSDAQ의 종목 데이터를 가져오거나 로컬 데이터프레임을 사용하여 관련된 업종 정보(업종, 시가총액, 거래량, 거래대금)를 결합합니다.

    Parameters:
    market (str, optional): 'KOSPI' 또는 'KOSDAQ' 중 하나를 선택하여 해당 시장의 데이터를 가져옵니다. 
                            이 매개변수가 제공되지 않으면 local_df가 사용됩니다.
    local_df (pd.DataFrame, optional): 로컬에서 가져온 종목 데이터프레임. 
                                        제공되지 않으면 market 매개변수에 따라 데이터를 가져옵니다.
    local_market (str, optional): 제공된 local_df가 어떤 시장의 데이터인지 지정합니다. 
                                   'KOSPI' 또는 'KOSDAQ' 중 하나여야 합니다.

    Raises:
    ValueError: 
        - market 또는 local_df 중 하나는 반드시 제공되어야 합니다.
        - local_market은 'KOSPI' 또는 'KOSDAQ'이어야 합니다.
        - 지원하지 않는 시장이 제공된 경우.

    Returns:
    pd.DataFrame: Sector, Volume, Marcap, Amount 정보가 추가된 종목 데이터프레임.

    Example:
    >>> df_with_sector = add_sector_info(market='KOSPI')
    >>> df_with_local = add_sector_info(local_df=my_local_df, local_market='KOSDAQ')
    '''
    
    if local_df is None and market is None:
        raise ValueError("market 또는 local_df중 하나는 반드시 제공되어야 합니다.")
    
    # 종목 데이터 가져오기
    
    # 전처리 된 로컬 파일을 가져와서 쓰는경우
    elif local_df is not None :
        df = local_df
        
    # 미리 작업한 파일이 없는경우  
    elif market in ['KOSPI','KOSDAQ'] :
        df = combined_stock_analysis(market=market, month_ago=1)
        
    else:
        raise ValueError(" 'KOSPI', 'KOSDAQ' 두 시장만 지원하는 기능입니다.")  
        
    # KOSPI 또는 KOSDAQ 데이터프레임 선택
    
    # 전처리 된 로컬 파일을 사용하는경우 : 그 파일이 어느 시장데이터를 전처리했는지 매개변수에 기입입
    if local_market is not None:
        sector_data = kospi if local_market.lower() == 'kospi' else kosdaq if local_market.lower() == 'kosdaq' else None
        if sector_data is None:
            raise ValueError("local_market은 'KOSPI' 또는 'KOSDAQ'이어야 합니다.")
    
    # 미리 작업한 파일이 없는경우 : market 매개변수에서 시장 종류 파악가능능
    elif market == 'KOSPI':
        sector_data = kospi
    elif market == 'KOSDAQ':
        sector_data = kosdaq

        
    # 'Code'를 인덱스로 설정
    sector_data = sector_data.set_index('Code')

    # 업종과 시가총액, 거래량, 거래대금 추가
    df = df.merge(sector_data[['Sector','Volume','Amount','Marcap']], left_on='Code', right_index=True, how='left')
    
    # kospi_return_add_sector.to_csv('new_KOSPI_add_sector_df.csv', index=False, encoding='utf-8-sig')
    # kosdaq_return_add_sector.to_csv('new_KOSDAQ_add_sector_df.csv', index=False, encoding='utf-8-sig')

    return df